In [0]:
from textgenrnn import textgenrnn

Using TensorFlow backend.


For Error refer to:
https://stackoverflow.com/questions/60581677/attributeerror-module-tensorflow-core-api-v2-config-has-no-attribute-expe

In [0]:
textgen = textgenrnn()

In [0]:
textgen.train_from_file('summary.txt', num_epochs=1)
textgen.generate()

6 texts collected.
Training on 475 character sequences.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
3/3 [==============================] - 2s 734ms/step - loss: 2.2802
####################
Temperature: 0.2
####################
Andr she dence hands, and meets and meat and and meet message and meet and meat and meat and meat and meat laps lack and meet and meet and meet and meet and meat and meat and meet and meet and melaati and meet and meet and meat and meas and meat and meat and meet and meet and meet and meat the day

And man hat same she lack and meat and meet and melaat and day and meet and meat the maid and meet and meet and meam and meet and meat and andineges, and meet and measion and meet and meet and meet and meat the day same and meat and andineges, and meet and meet and meet and meat and meat and meet 

Andr lanch hass says her did days, and meet my hands, and meet and meet lack and meet message message medies, and meet and meet and meat and meat and meat and meat and meat and meet and meet message the the the day and meet and meet and meet and meat and meet and

code taken from https://colab.research.google.com/drive/1mMKGnVxirJnqDViH7BDJxFqWrsXlPSoK

In [0]:
from datetime import datetime
import os
from google.colab import files

In [0]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 500000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 5,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [0]:
file_name = "horror10.txt"
model_name = 'colaboratory' 

In [0]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 3-layer, 128-cell LSTMs
Training on 805,145 character sequences.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
786/786 [==============================] - 1946s 2s/step - loss: 3.0273
Epoch 2/5
786/786 [==============================] - 1929s 2s/step - loss: 3.0072
Epoch 3/5
786/786 [==============================] - 1935s 2s/step - loss: 2.2418
Epoch 4/5
786/786 [==============================] - 1930s 2s/step - loss: 1.3088
Epoch 5/5
786/786 [==============================] - 1932s 2s/step - loss: 1.1774
####################
Temperature: 0.2
####################
ater, and did be the strange and the strange and dreadful that I did be the strange and the strange and the Maid, and to have a strange and strange and dreadful that did be a little that she did be a little the more to the Maid, and the Maid did be the strange and dreadful that I had not to be the s

he water, and the Maid did be somewhat that I did be all the strange and dreadful that did be a little with a strange and dreadful that I had not to be the strange and dreadful that did be a wondrous and dear the strange and stra

In [0]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)